## Importing Libraries

In [205]:
import requests
import re
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import datetime
import pandas as pd
import time
import concurrent.futures
import modules.psql as psql

## Postgres Configuration

In [206]:
%run config_psql.ipynb

## Setting Configurations

In [207]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None  # default='warn'

## Initializing Parameters

In [195]:
min_year = 2004 #Parameterize this for receiving future incremental links
max_year = 2025 #datetime.date.today().year

df_master_url = pd.DataFrame(columns=['url_type','season','series','match_id','url'])

# test_series = [["2023%2F24", "ICC Cricket World Cup", "https://www.espncricinfo.com/series/icc-cricket-world-cup-2023-24-1367856/match-schedule-fixtures"]]

## Finding all Seasons

In [196]:
for year in range(min_year,max_year+1):
    new_row = {
        'url_type': 'season',
        'season': str(year),
        'url': "https://www.espncricinfo.com/ci/engine/series/index.html?season={};view=season".format(str(year))
    }
    df_master_url.loc[len(df_master_url)] = new_row
    
    if year < max_year:
        season_name = str(year) + "%2F" + '{:02d}'.format(year-1999)
        new_row = {
            'url_type': 'season',
            'season': season_name,
            'url': "https://www.espncricinfo.com/ci/engine/series/index.html?season={};view=season".format(season_name)
        }
        df_master_url.loc[len(df_master_url)] = new_row
df_master_url.head()

,url_type,season,series,match_id,url
0,season,2004,NaN,NaN,https://www.espncricinfo.com/ci/engine/series/...
1,season,2004%2F05,NaN,NaN,https://www.espncricinfo.com/ci/engine/series/...
2,season,2005,NaN,NaN,https://www.espncricinfo.com/ci/engine/series/...
3,season,2005%2F06,NaN,NaN,https://www.espncricinfo.com/ci/engine/series/...
4,season,2006,NaN,NaN,https://www.espncricinfo.com/ci/engine/series/...


In [197]:
df_url = df_master_url[df_master_url['season'].str.contains('2024')].reset_index()
df_url.drop(['index'], axis=1, inplace=True)
df_url

,url_type,season,series,match_id,url
0,season,2024,NaN,NaN,https://www.espncricinfo.com/ci/engine/series/...
1,season,2024%2F25,NaN,NaN,https://www.espncricinfo.com/ci/engine/series/...


## Finding all series in a season

In [198]:
for season in df_url['season'].values:
    season_engine_url = "https://www.espncricinfo.com/ci/engine/series/index.html?season={};view=season".format(season)
    response = requests.get(season_engine_url)
    soup = BeautifulSoup(response.content, 'lxml')
    for a in soup.find_all('a', href = True):
        # print(a)
        if (('https://www.espncricinfo.com/series/' in a['href']) and ('/match-schedule-fixtures' in a['href'])):
            new_row = {
                'url_type': 'series',
                'season': season,
                'series': a.text,
                'url': a['href']
            }
            df_url.loc[len(df_url)] = new_row
df_url.head()

,url_type,season,series,match_id,url
0,season,2024,NaN,NaN,https://www.espncricinfo.com/ci/engine/series/...
1,season,2024%2F25,NaN,NaN,https://www.espncricinfo.com/ci/engine/series/...
2,series,2024,ICC World Test Championship,NaN,https://www.espncricinfo.com/series/icc-world-...
3,series,2024,Botham-Richards Trophy,NaN,https://www.espncricinfo.com/series/botham-ric...
4,series,2024,Sri Lanka in England Test Series,NaN,https://www.espncricinfo.com/series/sri-lanka-...


## Finding all matches in a series using Multi-Threading

### Function to find all urls

In [199]:
def insert_match_url(season, series, response):
    new_rows = []
    soup = BeautifulSoup(response.content, 'lxml')
    divs = soup.find_all('div', class_ = 'ds-p-0') # if this doesn't work use class_ = ds-relative
    for div in divs:
        for a in div.find_all('a', href = True):
            if (('/live-cricket-score' in a['href']) or ('/full-scorecard' in a['href'])):
                new_row = {
                    'url_type': 'match',
                    'season': season,
                    'series': series,
                    'match_id': a['href'].split('/')[-2].split('-')[-1],
                    'url': 'https://www.espncricinfo.com' + a['href']
                }
                new_rows.append(new_row)
    return new_rows

### Function to process series urls to match urls

In [200]:
def process_url(season, series, url):
    st = time.time()
    initial_size = len(df_url)
    link1 = url.replace(url.split('/')[-1], 'match-results') # completed series
    link2 = url.replace(url.split('/')[-1], 'match-schedule-fixtures') # ongoing, future series
    link3 = url.replace(url.split('/')[-1], 'match-schedule-results') # alternate link
    response1 = requests.get(link1)
    response2 = requests.get(link2)
    response3 = requests.get(link3)
    if response1.status_code != 404:
        new_rows = insert_match_url(season, series, response1)
    elif response2.status_code != 404:
        new_rows = insert_match_url(season, series, response2)
    elif response3.status_code != 404:
        new_rows = insert_match_url(season, series, response3)
    else:
        print("No suitable match endpoint found, Intervention required!!!")
        print(season, series, url, sep='\n', end='\n\n')
    return new_rows

### Multi-Threading function

In [201]:
num_threads = 50
num_series = len(df_url[df_url['url_type'] == 'series'])

print("There are {} series to be processed".format(num_series))
print("wait for approx {} minutes".format((1.3*num_series//60)+1))
with concurrent.futures.ThreadPoolExecutor(max_workers = num_threads) as executor:
    futures = []
    for season, series, url in df_url[df_url['url_type'] == 'series'][['season', 'series', 'url']][:300].values:
    #for season, series, url in test_series:    
        futures.append(executor.submit(process_url, season, series, url))

    for count, future in enumerate(concurrent.futures.as_completed(futures)):
        for row in future.result():
            df_url.loc[len(df_url)] = row
            print("Season - {} | Series - {} | Match - {}".format(row['season'], row['series'], row['match_id']))
            # if (len(df_url)%100)==0:
            #     print("{} series added!".format(count))

There are 34 series to be processed
wait for approx 1.0 minutes
Season - 2024 | Series - Botham-Richards Trophy | Match - 1385691
Season - 2024 | Series - Botham-Richards Trophy | Match - 1385692
Season - 2024 | Series - Botham-Richards Trophy | Match - 1385693
Season - 2024 | Series - Sri Lanka in England Test Series | Match - 1385694
Season - 2024 | Series - Sri Lanka in England Test Series | Match - 1385695
Season - 2024 | Series - Sri Lanka in England Test Series | Match - 1385696
Season - 2024 | Series - ICC World Test Championship | Match - 1336043
Season - 2024 | Series - ICC World Test Championship | Match - 1336044
Season - 2024 | Series - ICC World Test Championship | Match - 1336045
Season - 2024 | Series - ICC World Test Championship | Match - 1381212
Season - 2024 | Series - ICC World Test Championship | Match - 1385685
Season - 2024 | Series - ICC World Test Championship | Match - 1336046
Season - 2024 | Series - ICC World Test Championship | Match - 1381213
Season - 2024

In [202]:
query = psql.insert_without_duplicate(
    engine,
    dataFrame = df_url,
    table = "espn_url",
    schema = "dwh",
    conflict_col = list(df_url.columns))

In [23]:
df_url[df_url['url_type'] == 'series'][['season', 'series', 'url']][300:500].values

array([['2018%2F19', 'South Africa in Australia T20I Match',
        'https://www.espncricinfo.com/series/south-africa-in-australia-t20i-match-2018-19-1144970/match-schedule-fixtures'],
       ['2018%2F19', 'India in Australia T20I Series',
        'https://www.espncricinfo.com/series/india-in-australia-t20i-series-2018-19-1144972/match-schedule-fixtures'],
       ['2018%2F19', 'West Indies in Bangladesh T20I Series',
        'https://www.espncricinfo.com/series/west-indies-in-bangladesh-t20i-series-2018-19-1153309/match-schedule-fixtures'],
       ['2018%2F19', 'Sri Lanka in New Zealand T20I Match',
        'https://www.espncricinfo.com/series/sri-lanka-in-new-zealand-t20i-match-2018-19-1153837/match-schedule-fixtures'],
       ['2018%2F19', 'ACC Western Region T20',
        'https://www.espncricinfo.com/series/acc-western-region-t20-2018-19-1171739/match-schedule-fixtures'],
       ['2018%2F19', 'Nepal in United Arab Emirates T20I Series',
        'https://www.espncricinfo.com/series